In [1]:
import numpy as np
import pandas as pd

# stuck with the CPU version for now...
high_exp = pd.read_csv("high_exp.csv", index_col=0)
low_exp = pd.read_csv("low_exp.csv", index_col=0)

data_df = pd.concat([high_exp, low_exp], axis=0)

def string_to_matrix(color_string):
    color_string = str(color_string)
    list_of_strings = color_string.replace('[', '').replace(']', '').split('\n')
    list_of_lists = [channels.strip().split() 
                     for channels in list_of_strings
                     if 'nan' not in list_of_strings
                    ]
    remaining_pad = 181 - len(list_of_lists)
    while remaining_pad > 0:
        list_of_lists.append(list([0, 0, 0, 0]))
        remaining_pad = remaining_pad - 1
    
    return np.array(list_of_lists).astype(np.float)

data_df['color_matrix'] = data_df['color_matrix'].apply(string_to_matrix)

# create train test split
from sklearn.model_selection import train_test_split

X = np.zeros((22604, 181, 4))
for idx, colors in enumerate(data_df['color_matrix'].values):
    X[idx, :, :] = colors
    
y = data_df['class'].values

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(100, 10, activation='relu', input_shape=(181, 4)))
model.add(MaxPooling1D(pool_size=10))
model.add(Flatten())
# model.add(Dense(32))
# model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Using TensorFlow backend.


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [3]:
model.fit(x_train, y_train, batch_size=50, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=50)

Epoch 1/10
15822/15822 [==============================] - 2s 156us/step - loss: 0.6928 - acc: 0.5212
Epoch 2/10
15822/15822 [==============================] - 2s 147us/step - loss: 0.6884 - acc: 0.5370
Epoch 3/10
15822/15822 [==============================] - 2s 147us/step - loss: 0.6861 - acc: 0.5480
Epoch 4/10
15822/15822 [==============================] - 2s 150us/step - loss: 0.6827 - acc: 0.5611
Epoch 5/10
15822/15822 [==============================] - 2s 147us/step - loss: 0.6781 - acc: 0.5714
Epoch 6/10
15822/15822 [==============================] - 2s 148us/step - loss: 0.6735 - acc: 0.5842
Epoch 7/10
15822/15822 [==============================] - 2s 146us/step - loss: 0.6683 - acc: 0.5955
Epoch 8/10
15822/15822 [==============================] - 2s 147us/step - loss: 0.6631 - acc: 0.6061
Epoch 9/10
15822/15822 [==============================] - 2s 148us/step - loss: 0.6577 - acc: 0.6165
Epoch 10/10
6782/6782 [==============================] - 0s 49us/step
